In [1]:
from __future__ import division
import os
import sys
import glob
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
%matplotlib inline
%precision 4
plt.style.use('ggplot')


In [2]:
from numba import jit, int32, int64, float32, float64 

In [3]:
%load_ext cythonmagic

Converting Python Code to C for speed
====

## Example: Fibonacci

In [4]:
def py_fib(n):
    a, b = 0, 1
    for i in range(n):
        a, b = a+b, a
    return a

In [5]:
@jit(float64(int64), nopython=True, locals={'a': float64})
def numba_fib(n):
    a = 0
    b = 1
    for i in range(n):
        tmp = a
        a = a + b
        b = tmp
    return a

In [6]:
%%cython -a
def cy_fib(n):
    a = 0
    b = 1
    for i in range(n):
        a, b = a+b, a
    return a

In [7]:
%%cython -a
cpdef double cy_fib(int n):
    cdef double a, b
    a = 0
    b = 1
    for i in range(n):
        a, b = a+b, a
    return a

In [8]:
%timeit py_fib(100)
%timeit numba_fib(100)
%timeit cy_fib(100)

100000 loops, best of 3: 8.77 µs per loop
1000000 loops, best of 3: 475 ns per loop
1000000 loops, best of 3: 250 ns per loop


## Example: Matrix multiplication

In [9]:
%%cython -a

import numpy as np
cimport numpy as np

def py_mult(u, v):
    m, n = u.shape
    n, p = v.shape
    w = np.zeros((m, p))
    for i in range(m):
        for j in range(p):
            for k in range(n):
                w[i, j] += u[i, k] * v[k, j]
    return w

In [10]:
@jit
def numba_mult(u, v):
    m, n = u.shape
    n, p = v.shape
    w = np.zeros((m, p))
    for i in range(m):
        for j in range(p):
            for k in range(n):
                w[i, j] += u[i, k] * v[k, j]
    return w   

In [11]:
%%cython -a
import numpy as np
cimport numpy as np

cimport cython
@cython.boundscheck(False)
@cython.wraparound(False)
def cy_mult(double[:,::1] u, double[:,::1] v):
    m = u.shape[0]
    n = u.shape[1]
    p = v.shape[1]
    
    cdef int i, j, k
    cdef double[:,::1] w = np.zeros((m, p), dtype=np.float64)
    for i in range(m):
        for j in range(p):
            for k in range(n):
                w[i, j] += u[i, k] * v[k, j]
    return np.asarray(w)

In [12]:
m = 50
n = 30
p = 40
u = np.random.random((m, n))
v = np.random.random((n, p))

In [13]:
%timeit -n 10 np.dot(u, v)
%timeit -n 10 py_mult(u, v)
%timeit -n 10 numba_mult(u, v)
%timeit -n 10 cy_mult(u, v)

10 loops, best of 3: 16 µs per loop
10 loops, best of 3: 46 ms per loop
10 loops, best of 3: 74.2 µs per loop
10 loops, best of 3: 205 µs per loop


## Example: Pairwise distance matrix

#### Python

In [14]:
def py_euclidean(vs, i, j):
    n = vs.shape[1]
    d = 0.0
    for k in range(n):
        t = vs[i, k] - vs[j, k]
        d += t*t
    return np.sqrt(d)

def py_pairwise(vs, dist):
    n = len(vs)
    ds = np.empty((n, n))
    for i in range(n):
        for j in range(n):
            if i == j:
                ds[i, j] = 0
            elif i > j:
                ds[i, j] = ds[j, i]
            else:
                ds[i, j] = dist(vs, i, j)
    return ds

In [15]:
vs = np.random.random((100, 100))

In [16]:
py_pairwise(vs, py_euclidean)

array([[ 0.    ,  4.0287,  4.0001, ...,  4.3067,  4.0137,  4.2026],
       [ 4.0287,  0.    ,  3.7854, ...,  3.7035,  3.8681,  4.2034],
       [ 4.0001,  3.7854,  0.    , ...,  4.0247,  4.2126,  4.0511],
       ..., 
       [ 4.3067,  3.7035,  4.0247, ...,  0.    ,  3.8594,  4.4682],
       [ 4.0137,  3.8681,  4.2126, ...,  3.8594,  0.    ,  4.1816],
       [ 4.2026,  4.2034,  4.0511, ...,  4.4682,  4.1816,  0.    ]])

## Profiling code

In [17]:
%timeit py_pairwise(vs, py_euclidean)

1 loops, best of 3: 520 ms per loop


In [18]:
prof = %prun -r -q py_pairwise(vs, py_euclidean)

In [19]:
prof.sort_stats('time').print_stats(10);

         10006 function calls in 0.569 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
     4950    0.551    0.000    0.559    0.000 <ipython-input-14-bdacffbdf484>:1(py_euclidean)
        1    0.010    0.010    0.569    0.569 <ipython-input-14-bdacffbdf484>:9(py_pairwise)
     5051    0.008    0.000    0.008    0.000 {range}
        1    0.000    0.000    0.000    0.000 {numpy.core.multiarray.empty}
        1    0.000    0.000    0.569    0.569 <string>:1(<module>)
        1    0.000    0.000    0.000    0.000 {len}
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}




## Numba

In [20]:
@jit(nopython=True)
def numba_euclidean(vs, i, j):
    n = vs.shape[1]
    d = 0.0
    for k in range(n):
        t = vs[i, k] - vs[j, k]
        d += t*t
    return np.sqrt(d)

@jit
def numba_pairwise(vs, dist):
    n = vs.shape[0]
    ds = np.empty((n, n))
    for i in range(n):
        for j in range(n):
            if i == j:
                ds[i, j] = 0
            elif i > j:
                ds[i, j] = ds[j, i]
            else:
                ds[i, j] = dist(vs, i, j)
    return ds

In [21]:
%timeit -n 100 py_euclidean(vs, 0, 1)
%timeit -n 100 numba_euclidean(vs, 0, 1)

100 loops, best of 3: 111 µs per loop
100 loops, best of 3: 1.12 µs per loop


In [22]:
%timeit -n 10 py_pairwise(vs, py_euclidean)
%timeit -n 10 numba_pairwise(vs, numba_euclidean)

10 loops, best of 3: 537 ms per loop
10 loops, best of 3: 6.44 ms per loop


In [23]:
prof = %prun -r -q numba_pairwise(vs, numba_euclidean)

In [24]:
prof.sort_stats('time').print_stats(10);

         28 function calls in 0.014 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.014    0.014    0.014    0.014 <string>:1(<module>)
        2    0.000    0.000    0.000    0.000 numpy_support.py:131(map_arrayscalar_type)
        2    0.000    0.000    0.000    0.000 numpy_support.py:80(from_dtype)
        2    0.000    0.000    0.000    0.000 context.py:130(resolve_data_type)
       16    0.000    0.000    0.000    0.000 {isinstance}
        2    0.000    0.000    0.000    0.000 dispatcher.py:178(typeof_pyval)
        2    0.000    0.000    0.000    0.000 numpy_support.py:144(is_array)
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}




## Cython

In [25]:
%%cython -a -lm
from libc.math cimport sqrt 
import numpy as np
cimport numpy as np
cimport cython

# function pointer 
ctypedef double (*func)(double[:, ::1], int i, int j)

@cython.boundscheck(False)
@cython.wraparound(False)
cdef double cy_euclidean(double[:, ::1] vs, int i, int j):
    cdef double d = 0.0, t
    cdef int k

    cdef int n = vs.shape[1]
    for k in range(n):
        t = vs[i, k] - vs[j, k]
        d += t*t
    return sqrt(d)

@cython.boundscheck(False)
@cython.wraparound(False)
def cy_pairwise(double[:,::1] vs, metric='euclidean'):
    n = vs.shape[0]
    cdef double[:, ::1] ds = np.empty((n, n))
    cdef int i, j

    cdef func dist
    if metric == 'euclidean':
        dist = &cy_euclidean
    else:
        raise ValueError('Metric not found')

    for i in range(n):
        for j in range(n):
            if i == j:
                ds[i, j] = 0
            elif i > j:
                ds[i, j] = ds[j, i]
            else:
                ds[i, j] = dist(vs, i, j)
    return np.asarray(ds)

In [26]:
%timeit -n 10 py_pairwise(vs, py_euclidean)
%timeit -n 10 numba_pairwise(vs, numba_euclidean)
%timeit -n 10 cy_pairwise(vs)

10 loops, best of 3: 576 ms per loop
10 loops, best of 3: 7.2 ms per loop
10 loops, best of 3: 640 µs per loop


## Comparison with optimized C from scipy

In [27]:
from scipy.spatial.distance import pdist

%timeit -n 10 pdist(vs)

10 loops, best of 3: 626 µs per loop
